# Data Drift Workflow



In [1]:
import nannyml as nml
import pandas as pd
import numpy as np

# not sure how to further expose it as nml.datasets.load_synthetic_sample
from nannyml.datasets import load_synthetic_sample

In [2]:
reference, analysis, analysis_gt = load_synthetic_sample()

In [3]:
features_for_drift = list(reference.columns)[:7] + ['y_pred_proba']
features_for_drift

['distance_from_office',
 'salary_range',
 'gas_price_per_litre',
 'public_transportation_cost',
 'wfh_prev_workday',
 'workday',
 'tenure',
 'y_pred_proba']

In [4]:
md = nml.extract_metadata(data = reference, model_name='wfh_predictor')

/var/home/nikml/Source/nannyml/nannyml/metadata.py:420: UserWarning: NannyML extracted 4 categorical features.
Please review these to determine if they should be marked as ordinal instead.

  warnings.warn(


In [5]:
print(md)

Metadata for model wfh_predictor

# Warning - unable to identify all essential data
# Please identify column names for all '~ UNKNOWN ~' values

Model problem             binary_classification    

Identifier column         identifier               
Timestamp column          ~ UNKNOWN ~              
Partition column          partition                
Prediction column         ~ UNKNOWN ~              
Ground truth column       ~ UNKNOWN ~              

Features

Name                 Column               Type            Description
distance_from_office distance_from_office continuous      extracted feature: distance_from_office
salary_range         salary_range         categorical     extracted feature: salary_range
gas_price_per_litre  gas_price_per_litre  continuous      extracted feature: gas_price_per_litre
public_transportation_cost public_transportation_cost continuous      extracted feature: public_transportation_cost
wfh_prev_workday     wfh_prev_workday     categorical     ex

In [6]:
md.prediction_column_name = 'y_pred_proba'
md.ground_truth_column_name = 'work_home_actual'
md.timestamp_column_name = 'DATE'

In [7]:
print(md)

Metadata for model wfh_predictor

# Warning - unable to identify all essential data
# Please identify column names for all '~ UNKNOWN ~' values

Model problem             binary_classification    

Identifier column         identifier               
Timestamp column          DATE                     
Partition column          partition                
Prediction column         y_pred_proba             
Ground truth column       work_home_actual         

Features

Name                 Column               Type            Description
distance_from_office distance_from_office continuous      extracted feature: distance_from_office
salary_range         salary_range         categorical     extracted feature: salary_range
gas_price_per_litre  gas_price_per_litre  continuous      extracted feature: gas_price_per_litre
public_transportation_cost public_transportation_cost continuous      extracted feature: public_transportation_cost
wfh_prev_workday     wfh_prev_workday     categorical     ex

In [8]:
univariate_calculator = nml.StatisticalDriftCalculator(model_metadata=md, features=features_for_drift)

In [9]:
univariate_calculator.fit(reference_data=reference)

In [10]:
fdata = pd.concat([reference, analysis])
fdata

,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,identifier,DATE,work_home_actual,y_pred_proba,partition
0,5.962247,40K - 60K €,2.119485,8.568058,False,Friday,0.212653,0,2015-05-06 13:12:20,1.0,0.99,reference
1,0.535872,40K - 60K €,2.357199,5.425382,True,Tuesday,4.927549,1,2020-08-05 14:58:14,0.0,0.07,reference
2,1.969519,40K - 60K €,2.366849,8.247158,False,Monday,0.520817,2,2014-03-02 06:31:16,1.0,1.00,reference
3,2.530410,20K - 20K €,2.318722,7.944251,False,Tuesday,0.453649,3,2020-12-14 18:06:37,1.0,0.98,reference
4,2.253635,60K+ €,2.221265,8.884478,True,Thursday,5.695263,4,2018-06-09 13:16:14,1.0,0.99,reference
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,6.043911,0 - 20K €,1.983026,5.891216,True,Thursday,6.411578,99995,2020-03-01 06:02:02,NaN,0.17,analysis
49996,5.676665,20K - 20K €,2.048549,7.584098,True,Wednesday,3.863509,99996,2014-08-31 12:20:36,NaN,0.55,analysis
49997,3.143112,0 - 20K €,2.208197,6.574665,True,Tuesday,6.462975,99997,2011-04-30 06:32:24,NaN,0.22,analysis
49998,8.335141,40K - 60K €,2.394480,5.257455,True,Monday,6.407057,99998,2018-02-02 05:56:29,NaN,0.02,analysis


In [11]:
# Hotfix because currently drift calculator has a bug and uses gt from analysis that is NaN
hotfix = fdata.copy(deep=True)
hotfix['work_home_actual'] = np.random.randint(0, 2, size=hotfix.shape[0])

In [12]:
univariate_results = univariate_calculator.calculate(data=hotfix, chunk_size=5000)

In [13]:
list(univariate_results.columns)

['key',
 'start_index',
 'end_index',
 'start_date',
 'end_date',
 'partition',
 'salary_range_chi2',
 'salary_range_p_value',
 'salary_range_alert',
 'wfh_prev_workday_chi2',
 'wfh_prev_workday_p_value',
 'wfh_prev_workday_alert',
 'workday_chi2',
 'workday_p_value',
 'workday_alert',
 'public_transportation_cost_dstat',
 'public_transportation_cost_p_value',
 'public_transportation_cost_alert',
 'tenure_dstat',
 'tenure_p_value',
 'tenure_alert',
 'distance_from_office_dstat',
 'distance_from_office_p_value',
 'distance_from_office_alert',
 'gas_price_per_litre_dstat',
 'gas_price_per_litre_p_value',
 'gas_price_per_litre_alert']

In [14]:
univariate_results

,key,start_index,end_index,start_date,end_date,partition,salary_range_chi2,salary_range_p_value,salary_range_alert,wfh_prev_workday_chi2,...,public_transportation_cost_alert,tenure_dstat,tenure_p_value,tenure_alert,distance_from_office_dstat,distance_from_office_p_value,distance_from_office_alert,gas_price_per_litre_dstat,gas_price_per_litre_p_value,gas_price_per_litre_alert
0,[0:4999],0,4999,2011-01-02,2020-12-31 23:59:59,reference,2.898781,0.407,False,0.414606,...,False,0.00978,0.774,False,0.01034,0.712,False,0.01122,0.612,False
1,[5000:9999],5000,9999,2011-01-01,2020-12-31 23:59:59,reference,3.144391,0.370,False,0.033486,...,False,0.01192,0.534,False,0.00750,0.959,False,0.01222,0.502,False
2,[10000:14999],10000,14999,2011-01-01,2020-12-30 23:59:59,reference,2.451881,0.484,False,0.168656,...,False,0.01268,0.454,False,0.00820,0.917,False,0.00886,0.865,False
3,[15000:19999],15000,19999,2011-01-01,2020-12-31 23:59:59,reference,4.062620,0.255,False,0.056270,...,False,0.01074,0.667,False,0.00860,0.887,False,0.00956,0.797,False
4,[20000:24999],20000,24999,2011-01-03,2020-12-31 23:59:59,reference,2.413988,0.491,False,0.242059,...,False,0.00924,0.829,False,0.00910,0.842,False,0.00758,0.955,False
5,[25000:29999],25000,29999,2011-01-01,2020-12-30 23:59:59,reference,3.796063,0.284,False,3.614573,...,False,0.00794,0.935,False,0.01458,0.286,False,0.01032,0.714,False
6,[30000:34999],30000,34999,2011-01-02,2020-12-28 23:59:59,reference,3.228836,0.358,False,0.075705,...,False,0.01120,0.615,False,0.01290,0.432,False,0.01094,0.644,False
7,[35000:39999],35000,39999,2011-01-02,2021-01-01 23:59:59,reference,1.393300,0.707,False,0.414606,...,False,0.00740,0.963,False,0.01380,0.349,False,0.01736,0.128,False
8,[40000:44999],40000,44999,2011-01-01,2020-12-31 23:59:59,reference,0.304785,0.959,False,0.012656,...,False,0.01464,0.281,False,0.01586,0.201,False,0.00842,0.901,False
9,[45000:49999],45000,49999,2011-01-01,2020-12-31 23:59:59,reference,2.987581,0.394,False,2.203832,...,False,0.01306,0.417,False,0.00924,0.829,False,0.00786,0.939,False


In [15]:
ranking = nml.AlertCountRanking()

In [16]:
ranked_features_drifted = ranking.rank(univariate_results)

In [17]:
ranked_features_drifted

,feature,number_of_alerts,rank
0,salary_range,5,1
1,wfh_prev_workday,5,2
2,public_transportation_cost,5,3
3,distance_from_office,5,4
4,tenure,2,5
5,workday,0,6
6,gas_price_per_litre,0,7


In [18]:
nml.calculate_statistical_drift(reference_data=reference, analysis_data=analysis, model_metadata=md)

KeyError: 'work_home_actual'

In [19]:
rcerror_calculator = nml.ReconstructionErrorDriftCalculator(model_metadata=md
)

In [20]:
rcerror_calculator.fit(reference_data=reference)

In [21]:
rcerror_results = rcerror_calculator.calculate(data=fdata, chunk_size=5000)
rcerror_results

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
rcerror_results = rcerror_calculator.calculate(data=hotfix, chunk_size=5000)
rcerror_results

,key,start_index,end_index,start_date,end_date,partition,reconstruction_error,alert
0,[0:4999],0,4999,2011-01-02,2020-12-31 23:59:59,reference,1.120961,0
1,[5000:9999],5000,9999,2011-01-01,2020-12-31 23:59:59,reference,1.118071,0
2,[10000:14999],10000,14999,2011-01-01,2020-12-30 23:59:59,reference,1.117237,0
3,[15000:19999],15000,19999,2011-01-01,2020-12-31 23:59:59,reference,1.125514,0
4,[20000:24999],20000,24999,2011-01-03,2020-12-31 23:59:59,reference,1.109446,0
5,[25000:29999],25000,29999,2011-01-01,2020-12-30 23:59:59,reference,1.122759,0
6,[30000:34999],30000,34999,2011-01-02,2020-12-28 23:59:59,reference,1.107138,0
7,[35000:39999],35000,39999,2011-01-02,2021-01-01 23:59:59,reference,1.127134,0
8,[40000:44999],40000,44999,2011-01-01,2020-12-31 23:59:59,reference,1.114237,0
9,[45000:49999],45000,49999,2011-01-01,2020-12-31 23:59:59,reference,1.110450,0
